In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


## Read Input Data from parquet files

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, collect_list, size, col, row_number,sort_array
from pyspark.sql import Window

sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

graphPath = "/data/graphDFSample"
reversedGraph = sparkSession.read.parquet(graphPath)
reversedGraph.printSchema()

root
 |-- user: integer (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: integer (containsNull = true)



## Map Friend to User List
### Consider Friends connected to more than 1 user

In [11]:
friend_to_user = reversedGraph.withColumn("friend",explode("friends")) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("user_list")) \
    .filter(size("user_list")>1)
    .select(sort_array("user_list").alias("users"))

### UDF to pair of users connected to a friend

In [14]:
from pyspark.sql.types import ArrayType,IntegerType
from pyspark.sql.functions import udf

def emit_pairs(array):
    alen = len(array)
    ans = []
    for i in xrange(alen):
        for j in xrange(i+1,alen):
            ans.append([array[i],array[j]])
    return ans

pair_udf = udf(lambda x:emit_pairs(x),ArrayType(ArrayType(IntegerType())))

## Call UDF to list of pairs & explode list

In [17]:
answer = friend_to_user.select(pair_udf("users").alias("pairs")) \
    .withColumn("pair",explode("pairs")) 
answer = answer.select(answer["pair"].getItem(0).alias("friend1"),ans["pair"].getItem(1).alias("friend2"))

#### Count repeatation number of pairs i.e. Mutual friends of pair

In [43]:
answer.createTempView("answer")
sparkSession.sql("select friend1 ,friend2, count(1) as cnt from answer group by friend1 ,friend2 order by cnt").show(10)